# DecentNet from conv layer

    # additionally needed
    """
    
    position
    activated channels
    connection between channels
    
    """

    
        
        # this layer id
        layer_id = 0
        
        # within this layer, a whole filter can be deactivated
        # within a filter, single channels can be deactivated
        # within this layer, filters can be swapped
     
* pruning actually doesn"t work: https://discuss.pytorch.org/t/pruning-doesnt-affect-speed-nor-memory-for-resnet-101/75814   
* fine tune a pruned model: https://stackoverflow.com/questions/73103144/how-to-fine-tune-the-pruned-model-in-pytorch
* an actual pruning mechanism: https://arxiv.org/pdf/2002.08258.pdf

pip install:
    pytorch_lightning

In [6]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import math
import warnings

import torch
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn.modules.module import Module
from torch.nn.modules.utils import _single, _pair, _triple, _reverse_repeat_tuple
from torch._torch_docs import reproducibility_notes

from torch.nn.common_types import _size_1_t, _size_2_t, _size_3_t
from typing import Optional, List, Tuple, Union

import sys
sys.path.insert(0, "helper")
sys.path.insert(0, "/helper")
sys.path.insert(0, "./helper")
sys.path.insert(0, "../helper")
print(sys.path)

# own module
from visualisation.feature_map import *

import random

['../helper', './helper', '/helper', 'helper', 'C:\\Users\\Prinzessin\\projects\\decentnet\\datasceyence\\examples', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\python39.zip', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\DLLs', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta', '', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Prinzessin\\anaconda3\\envs\\feta\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Prinzessin\\.ipython']


# conv2d layer (slightly adapted original)

In [7]:
class _ConvNd(torch.nn.Module):

    __constants__ = ['stride', 'padding', 'dilation', 'groups',
                     'padding_mode', 'output_padding', 'in_channels',
                     'out_channels', 'kernel_size']
    __annotations__ = {'bias': Optional[torch.Tensor]}

    def _conv_forward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]) -> Tensor:
        ...

    in_channels: int
    _reversed_padding_repeated_twice: List[int]
    out_channels: int
    kernel_size: Tuple[int, ...]
    stride: Tuple[int, ...]
    padding: Union[str, Tuple[int, ...]]
    dilation: Tuple[int, ...]
    transposed: bool
    output_padding: Tuple[int, ...]
    groups: int
    padding_mode: str
    weight: Tensor
    bias: Optional[Tensor]

    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: Tuple[int, ...],
                 stride: Tuple[int, ...],
                 padding: Tuple[int, ...],
                 dilation: Tuple[int, ...],
                 transposed: bool,
                 output_padding: Tuple[int, ...],
                 groups: int,
                 bias: bool,
                 padding_mode: str,
                 device=None,
                 dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        print(factory_kwargs)
        super().__init__()
        if groups <= 0:
            raise ValueError('groups must be a positive integer')
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        valid_padding_strings = {'same', 'valid'}
        if isinstance(padding, str):
            if padding not in valid_padding_strings:
                raise ValueError(
                    "Invalid padding string {!r}, should be one of {}".format(
                        padding, valid_padding_strings))
            if padding == 'same' and any(s != 1 for s in stride):
                raise ValueError("padding='same' is not supported for strided convolutions")

        valid_padding_modes = {'zeros', 'reflect', 'replicate', 'circular'}
        if padding_mode not in valid_padding_modes:
            raise ValueError("padding_mode must be one of {}, but got padding_mode='{}'".format(
                valid_padding_modes, padding_mode))
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        self.output_padding = output_padding
        self.groups = groups
        self.padding_mode = padding_mode
        # `_reversed_padding_repeated_twice` is the padding to be passed to
        # `F.pad` if needed (e.g., for non-zero padding types that are
        # implemented as two ops: padding + conv). `F.pad` accepts paddings in
        # reverse order than the dimension.
        if isinstance(self.padding, str):
            self._reversed_padding_repeated_twice = [0, 0] * len(kernel_size)
            if padding == 'same':
                for d, k, i in zip(dilation, kernel_size,
                                   range(len(kernel_size) - 1, -1, -1)):
                    total_padding = d * (k - 1)
                    left_pad = total_padding // 2
                    self._reversed_padding_repeated_twice[2 * i] = left_pad
                    self._reversed_padding_repeated_twice[2 * i + 1] = (
                        total_padding - left_pad)
        else:
            self._reversed_padding_repeated_twice = _reverse_repeat_tuple(self.padding, 2)

        if transposed:
            self.weight = Parameter(torch.empty(
                (in_channels, out_channels // groups, *kernel_size), **factory_kwargs))
            #self.importance = Parameter(torch.empty(
            #    (in_channels, out_channels // groups), **factory_kwargs))
            
        else:
            self.weight = Parameter(torch.empty(
                (out_channels, in_channels // groups, *kernel_size), **factory_kwargs))
            #self.importance = Parameter(torch.empty(
            #    (out_channels, in_channels // groups), **factory_kwargs))
            
        if bias:
            self.bias = Parameter(torch.empty(out_channels, **factory_kwargs))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()


class CustomConv2d(_ConvNd):
    
    # additionally needed
    """
    
    position
    activated channels
    connection between channels
    
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: _size_2_t,
        stride: _size_2_t = 1,
        padding: Union[str, _size_2_t] = 0,
        dilation: _size_2_t = 1,
        groups: int = 1,
        bias: bool = True,
        padding_mode: str = 'zeros',  # TODO: refine this type
        device=None,
        dtype=None
    ) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        kernel_size_ = _pair(kernel_size)
        stride_ = stride #_pair(stride)
        padding_ = padding if isinstance(padding, str) else _pair(padding)
        dilation_ = _pair(dilation)
        super().__init__(
            in_channels, out_channels, kernel_size_, stride_, padding_, dilation_,
            False, _pair(0), groups, bias, padding_mode, **factory_kwargs)
        
        # this layer id
        layer_id = 0
        
        # within this layer, a whole filter can be deactivated
        # within a filter, single channels can be deactivated
        # within this layer, filters can be swapped

    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(k), 1/sqrt(k)), where k = weight.size(1) * prod(*kernel_size)
        # For more details see: https://github.com/pytorch/pytorch/issues/15314#issuecomment-477448573
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            if fan_in != 0:
                bound = 1 / math.sqrt(fan_in)
                init.uniform_(self.bias, -bound, bound)

    def extra_repr(self):
        s = ('{in_channels}, {out_channels}, kernel_size={kernel_size}'
             ', stride={stride}')
        if self.padding != (0,) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1,) * len(self.dilation):
            s += ', dilation={dilation}'
        if self.output_padding != (0,) * len(self.output_padding):
            s += ', output_padding={output_padding}'
        if self.groups != 1:
            s += ', groups={groups}'
        if self.bias is None:
            s += ', bias=False'
        if self.padding_mode != 'zeros':
            s += ', padding_mode={padding_mode}'
        return s.format(**self.__dict__)

    def __setstate__(self, state):
        super().__setstate__(state)
        if not hasattr(self, 'padding_mode'):
            self.padding_mode = 'zeros'

    def _conv_forward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]):
        if self.padding_mode != 'zeros':
            return F.conv2d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),
                            weight, bias, self.stride,
                            _pair(0), self.dilation, self.groups)
        
        # this is written in c++ - try not to change ...
        print(self.stride)
        return F.conv2d(input, weight, bias, self.stride,
                        self.padding, self.dilation, self.groups)

    def forward(self, input: Tensor) -> Tensor:
        return self._conv_forward(input, self.weight, self.bias)



# model

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = CustomConv2d(1, 32, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv2 = CustomConv2d(32, 64, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv3 = CustomConv2d(64, 128, kernel_size=3, stride=1, padding=0, dilation=3)
        self.conv1x1 = CustomConv2d(128, 10, kernel_size=1, stride=1, padding=0, dilation=1)
        
        self.K = 100 
        self.L = 10 # last one
        self.num_of_bases = 1 # 3rd dim
        
        if False:
            self.conv1 = Conv2d(1, 32, kernel_size=3, stride=1, padding=0, dilation=3)
            self.conv2 = Conv2d(32, 64, kernel_size=3, stride=1, padding=0, dilation=3)
            self.conv3 = Conv2d(64, 128, kernel_size=3, stride=1, padding=0, dilation=3)
            self.conv1x1 = Conv2d(128, 10, kernel_size=1, stride=1, padding=0, dilation=1)
        #self.dropout1 = nn.Dropout(0.25)
        #self.dropout2 = nn.Dropout(0.5)
        # 4x16384
        # self.fc1 = nn.Linear(10*10*10, 10)
        #self.fc2 = nn.Linear(10, 10)
        
        #self.flat = nn.Flatten()
        
        self.fc3 = nn.Linear(10, 10)
        
        self.mish1 = torch.nn.Mish()
        self.mish2 = torch.nn.Mish()
        self.mish3 = torch.nn.Mish()
        self.mish1x1 = torch.nn.Mish()
        
        #self.sub_concept_pooling = nn.modules.MaxPool2d((self.K, 1), stride=(1,1))
        #self.instance_pooling = nn.modules.MaxPool2d((opt.num_of_bases, 1), stride=(1,1))
        
        self.flat = nn.Flatten()
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        x = self.conv1(x)
        x = self.mish1(x)
        
        x = self.conv2(x)
        x = self.mish2(x)
        
        x = self.conv3(x)
        x = self.mish3(x)
        
        x = self.conv1x1(x)
        x = self.mish1x1(x)
        
        # print(x.shape)
        
        #x = F.max_pool2d(x, 2)
        #x = self.dropout1(x)
        
        #print(x.size())
        #print(x.size()[2:])
        
        x = F.avg_pool2d(x, kernel_size=x.size()[2:])
        
        x = x.reshape(x.size(0), -1)
        x = self.fc3(x)
        
        # x = self.flat(x)
        
        #x = self.fc2(x)
        
        
        #x = x.view(-1, self.L, self.K, 10)
        
        # input, kernel_size, stride, padding, dilation, ceil_mode
        #x = self.sub_concept_pooling(x).view(-1, self.L, self.num_of_bases).permute(0,2,1).unsqueeze(1)
        
        # output = F.sigmoid(x)
        # x = x.reshape(x.size(0), -1)
        
        #x = torch.flatten(x, 1)
        # x = self.fc1(x)
        #x = F.relu(x)
        
        #x = self.dropout2(x)
        #x = self.fc2(x)
        #output = F.log_softmax(x, dim=1)
        
        x = self.sigmoid(x)
        
        return x





# normal run

In [9]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for i_batch, (data, target) in enumerate(train_loader):
        
        
        
        data, target = data.to(device), target.unsqueeze(1) # .to(device)
        optimizer.zero_grad()
        output = model(data)
        
        target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
        
        if i_batch == -1:
            print(data.shape) # torch.Size([4, 1, 28, 28])
            print(target)
            """
            tensor([[8],
            [7],
            [2],
            [7]])
            """
            print(target_multi_hot)
            """
            tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
            [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')
            """
        
        loss = F.binary_cross_entropy(output, target_multi_hot)
        loss.backward()
        optimizer.step()
        if i_batch % (args.log_interval*1000) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i_batch * len(data), len(train_loader.dataset),
                100. * i_batch / len(train_loader), loss.item()))
            


def test(args, model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.unsqueeze(1) # .to(device)
            
            output = model(data)
            
            target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
            test_loss += F.binary_cross_entropy(output, target_multi_hot, reduction='mean').item()
        
            # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.to(device).view_as(pred)).sum().item()
            
            """
            if i == 0 and epoch % args.log_interval == 0:
            # if False: # i == 0:
                print(data.shape)
                layer = model.conv1x1 # model.conv1[0] # model.stage2[0].branch1[2] # model.fusion_layer # conv
                # run feature map
                dd = FeatureMap(model=model, layer=layer, device=device, iterations=None, lr=None)
                dd.run(data)
                dd.plot(path=f"example_results/feature_map_{epoch}.png")
                """

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

class Parser():
    def __init__(self):
        self.batch_size = 128
        self.test_batch_size = 1
        self.epochs = 100
        self.lr = 0.1
        self.gamma = 0.7
        self.log_interval = 5
        self.save_model = True
        

def main_train():
    # Training settings
    args = Parser()
    
    if True:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if device == torch.device("cuda"):
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('example_data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('example_data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr)

    #scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader, epoch)
        #scheduler.step()
        
        
        if args.save_model and epoch % args.log_interval == 0:
            torch.save(model.state_dict(), f"example_results/mnist_cnn_{epoch}.ckpt")


def main_test():
    # Training settings
    args = Parser()
    
    if True:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    test_kwargs = {'batch_size': args.test_batch_size}
    if device == torch.device("cuda"):
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset2 = datasets.MNIST('example_data', train=False,
                       transform=transform)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = Net().to(device)

    if True:
        model.load_state_dict(torch.load("example_results/mnist_cnn_5.ckpt"))
    else:
        model.load_state_dict(torch.load("example_results/pruned_model.ckpt"))
    

    # model = torch.load(model.state_dict(), "example_results/mnist_cnn_30.ckpt")
    if False:
        test(args, model, device, test_loader, 0)
    
    return model
        
    
            


In [5]:
# main_train()

In [6]:
# model_to_prune= main_test()

{'device': None, 'dtype': None}
{'device': None, 'dtype': None}
{'device': None, 'dtype': None}
{'device': None, 'dtype': None}


In [7]:
# print(list(model_to_prune.named_buffers()))

[]


# DecentNet trial and error

## DecentFilter
* conv2d problem: https://stackoverflow.com/questions/61269421/expected-stride-to-be-a-single-integer-value-or-a-list-of-1-values-to-match-the

In [10]:
class DecentFilter(torch.nn.Module):
    # convolution happens in here
    
    def __init__(self, 
                 i_channels=32, 
                 i_filter=[],
                 kernel_size=3, 
                 stride=1, 
                 padding=0, 
                 padding_mode="zeros",
                 dilation=3, 
                 transposed=None, 
                 device=None, 
                 dtype=None):
        
        #print("device", device)
        
        
        # out_channels = 1
        # groups = 1
        factory_kwargs = {'device': device, 'dtype': dtype}
        kernel_size = _pair(kernel_size)
        stride = stride # _pair(stride)
        padding = padding if isinstance(padding, str) else _pair(padding)
        dilation = _pair(dilation)
        
        
        valid_padding_strings = {'same', 'valid'}
        if isinstance(padding, str):
            if padding not in valid_padding_strings:
                raise ValueError(
                    "Invalid padding string {!r}, should be one of {}".format(
                        padding, valid_padding_strings))
            if padding == 'same' and any(s != 1 for s in stride):
                raise ValueError("padding='same' is not supported for strided convolutions")

        valid_padding_modes = {'zeros', 'reflect', 'replicate', 'circular'}
        if padding_mode not in valid_padding_modes:
            raise ValueError("padding_mode must be one of {}, but got padding_mode='{}'".format(
                valid_padding_modes, padding_mode))
            
        self.i_channels = i_channels
        self.i_filter = i_filter # index of this filter
        # self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        #self.output_padding = output_padding
        # self.groups = groups
        self.padding_mode = padding_mode
        
                
        
        
        # print(factory_kwargs)
        
        super().__init__()
    
        # filters x channels x kernel x kernel
        self.weight = Parameter(torch.empty((1, i_channels, *kernel_size), **factory_kwargs)) # .to(device)
        
        
        # set each channel true
        self.active = np.array([True] * self.i_channels)
        
        self.init_position()
        
        # for each filter, we need different true false vales for our channels
        #active = list(np.random.choice([True, False], size=w_channels, replace=True, p=None))

        # w_filters x w_channels x kernel x kernel
        #weights = torch.autograd.Variable(torch.randn(1,w_channels,3,3))

        
        
        #self.importance = Parameter(torch.empty(
        #    (in_channels), **factory_kwargs))
            
        if False: 
            # bias:
            # where should the bias be???
            self.bias = Parameter(torch.empty(1, **factory_kwargs))
        else:
            #self.bias = False
            self.register_parameter('bias', None)

        
        # reset weights and bias - in filter or in layer?
        self.reset_parameters()
        
    def init_position(self):
        # numbers fro 0 to 81
        # 0/0 to 
        self.position = list(np.random.randint(81, size=self.i_channels))
    
    def reset_parameters(self) -> None:
        # randomly initialise the positional array
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(k), 1/sqrt(k)), where k = weight.size(1) * prod(*kernel_size)
        # For more details see: https://github.com/pytorch/pytorch/issues/15314#issuecomment-477448573
        
        # todo - weight has to be the one in the filter
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
    
    def _conv_forward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]):
        
        
        
        if self.padding_mode != 'zeros':
            return F.conv2d(F.pad(input[:,self.active,:,:], self._reversed_padding_repeated_twice, mode=self.padding_mode),
                            weight, bias, self.stride,
                            _pair(0), self.dilation, groups=1)
        else:
            # this is written in c++
            # todo: cuda is ... not a variable ...
            out = F.conv2d(input[:,self.active,:,:], weight, bias, self.stride, self.padding, self.dilation, groups=1)
        
            # print(out.shape, "- batch x filters x width x height")        

            return out
        

    
    
    def update(self):
        # channel deactivation
        # require_grad = False/True for each channel
        
        deactivate_ids = [1, 2, 6]
        
        self.active[deactivate_ids] = False
        
        
        print("weight")
        print(self.weight.shape)
        
        print(self.weight[:,self.active,:,:].shape)
        
        # this is totally wrong - iterative will break after first iteration
        
        print()
        
        # Good to hear it’s working, although I would think you’ll get an error at some point in your code, as the cuda() call creates a non-leaf tensor.
        self.weight = torch.nn.Parameter(  self.weight[:,self.active,:,:] ) # .detach().cpu().numpy()
        self.weight = self.weight.cuda()
        
        
        print(self.weight.shape)
        #print(self.active)
        
    def forward(self, input: Tensor) -> Tensor:
        return self._conv_forward(input, self.weight, self.bias)
        
        

    
    
    

            


    

In [510]:
np.array([True] * 5)

array([ True,  True,  True,  True,  True])

In [511]:
l = torch.tensor([1, 6, 9, 40, 5, 3, 4]) # np.arange(50)
# l = l[[1, 6, 2, 4, 3]]

l[[1, 6, 2, 4, 3]] = False

l 
#np.where(l == 4) 

tensor([1, 0, 0, 0, 0, 3, 0])

## DecentLayer

In [11]:
class DecentLayer(torch.nn.Module):
    __constants__ = ['stride', 'padding', 'dilation', 'groups',
                     'padding_mode', 'in_channels', #  'output_padding',
                     'out_channels', 'kernel_size']
    __annotations__ = {'bias': Optional[torch.Tensor]}
        
        
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 kernel_size: _size_2_t,
                 stride: _size_2_t = 1,
                 padding: Union[str, _size_2_t] = 0,
                 dilation: _size_2_t = 1,
                 transposed: bool = False,
                 #output_padding: Tuple[int, ...] = _pair(0),
                 #groups: int = 1,
                 bias: bool = True,
                 padding_mode: str = "zeros",
                 device=None,
                 dtype=None) -> None:
        
        factory_kwargs = {'device': device, 'dtype': dtype}
        kernel_size = _pair(kernel_size)
        stride = stride # _pair(stride)
        padding = padding if isinstance(padding, str) else _pair(padding)
        dilation = _pair(dilation)
        
        super().__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # torch.nn.ModuleList
        
        self.geometry_array = np.full(81, np.nan)
        self.module_list = nn.ModuleList([])
        for i_filter in range(out_channels):
            self.module_list.append(DecentFilter(i_channels=in_channels, i_filter=i_filter))
            self.geometry_array[i_filter]=i_filter # id of the filter
        
        
        np.random.shuffle(self.geometry_array)
        self.geometry_array = self.geometry_array.reshape((9,9), order='C')
        
        # a = np.full(81, np.nan)
        # geometry_array = a.reshape((9,9))
        
        print()
        print(self.geometry_array)
        print()
        
        
        if bias:
            self.bias = Parameter(torch.empty(out_channels, **factory_kwargs))
        else:
            self.register_parameter('bias', None)

        # reset in initialisation
        self.reset_parameters()
        
        
    def sparcify(self) -> None:
        # pruning based on a metric
        
        # delete layer with id
        # delete channels in each layer with id
        
        # change positions
        # change
        
        pass
    
    def reset_parameters(self) -> None:
        pass
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(k), 1/sqrt(k)), where k = weight.size(1) * prod(*kernel_size)
        # For more details see: https://github.com/pytorch/pytorch/issues/15314#issuecomment-477448573
        
        # todo - weight has to be the one in the filter
        # init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        """
        not needed due to instance norm layer
        https://github.com/pytorch/vision/issues/4914
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            if fan_in != 0:
                bound = 1 / math.sqrt(fan_in)
                init.uniform_(self.bias, -bound, bound)"""

    def extra_repr(self):
        """
        s = ('{in_channels}, {out_channels}, kernel_size={kernel_size}'
             ', stride={stride}')
        if self.padding != (0,) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1,) * len(self.dilation):
            s += ', dilation={dilation}'
        if self.output_padding != (0,) * len(self.output_padding):
            s += ', output_padding={output_padding}'
        if self.groups != 1:
            s += ', groups={groups}'
        if self.bias is None:
            s += ', bias=False'
        if self.padding_mode != 'zeros':
            s += ', padding_mode={padding_mode}'
        return s.format(**self.__dict__)
        """
        return None

    def __setstate__(self, state):
        super().__setstate__(state)
        if not hasattr(self, 'padding_mode'):
            self.padding_mode = 'zeros'
            
    def update(self):
        for module in self.module_list:
            module.update()
        
            
            
    def forward(self, input: Tensor, pos=None) -> Tensor:
        
        output_list = []
        for module in self.module_list:
            output_list.append(module(input))
         
        out = torch.cat(output_list, dim=1)
        # mean = torch.mean(out, 0, keepdim=True)
        
        return out, pos
        # return self._conv_forward(input, self.weight, self.bias)
            

## DecentNet

In [12]:
class DecentNet(nn.Module):
    def __init__(self):
        super(DecentNet, self).__init__()
        
        dim = [1, 32, 48, 64, 10]
        dim = [1, 8, 16, 24, 10]
        assert not any(i > 81 for i in dim), "filters need to be less than 81"
        
        self.decent1 = DecentLayer(dim[0], dim[1], kernel_size=3, stride=1, padding=0, dilation=3)
        self.decent2 = DecentLayer(dim[1], dim[2], kernel_size=3, stride=1, padding=0, dilation=3)
        self.decent3 = DecentLayer(dim[2], dim[3], kernel_size=3, stride=1, padding=0, dilation=3)
        self.decent1x1 = DecentLayer(dim[3], dim[-1], kernel_size=1, stride=1, padding=0, dilation=1)

        self.fc3 = nn.Linear(dim[-1], dim[-1])
        
        self.mish1 = torch.nn.Mish()
        self.mish2 = torch.nn.Mish()
        self.mish3 = torch.nn.Mish()
        self.mish1x1 = torch.nn.Mish()
        
        self.bias1 = torch.nn.InstanceNorm2d(dim[1])
        self.bias2 = torch.nn.InstanceNorm2d(dim[2])
        self.bias3 = torch.nn.InstanceNorm2d(dim[3])
        self.bias1x1 = torch.nn.InstanceNorm2d(dim[-1])
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        x, pos = self.decent1(x)
        x = self.mish1(x)
        x = self.bias1(x)
        
        x, pos = self.decent2(x, pos)
        x = self.mish2(x)
        x = self.bias2(x)
        
        x, pos = self.decent3(x, pos)
        x = self.mish3(x)
        x = self.bias3(x)
        
        x, pos = self.decent1x1(x, pos)
        x = self.mish1x1(x)
        x = self.bias1x1(x)
        
        # global max pooling for MIL
        x = F.max_pool2d(x, kernel_size=x.size()[2:])
        
        x = x.reshape(x.size(0), -1)
        x = self.fc3(x) 
        
        x = self.sigmoid(x)
        
        return x
    
    def update(self):
        #self.decent1.update()
        self.decent2.update()
        self.decent3.update()
        self.decent1x1.update()
        


## standard routine

In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for i_batch, (data, target) in enumerate(train_loader):
        
        
        data, target = data.to(device), target.unsqueeze(1) # .to(device)
        optimizer.zero_grad()
        output = model(data)
        
        
        target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
        
        if i_batch == 5:
            
            
            #model.update()
            
            #print(data.shape) # torch.Size([4, 1, 28, 28])
            #print(target)
            """
            tensor([[8],
            [7],
            [2],
            [7]])
            """
            #print(target_multi_hot)
            """
            tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
            [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')
            """
            pass
        
        
        loss = F.binary_cross_entropy(output, target_multi_hot)
        loss.backward()
        optimizer.step()
        if i_batch % (args.log_interval*1000) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i_batch * len(data), len(train_loader.dataset),
                100. * i_batch / len(train_loader), loss.item()))
            
            # model.update()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.unsqueeze(1) # .to(device)
            
            output = model(data)
            
            target_multi_hot = torch.zeros(target.size(0), 10).scatter_(1, target, 1.).to(device)
            test_loss += F.binary_cross_entropy(output, target_multi_hot, reduction='mean').item()
        
            # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.to(device).view_as(pred)).sum().item()
            
            if False: # i == 0:
                print(data.shape)
                layer = model.conv1x1 # model.conv1[0] # model.stage2[0].branch1[2] # model.fusion_layer # conv
                # run feature map
                dd = FeatureMap(model=model, layer=layer, device=device, iterations=None, lr=None)
                dd.run(data)
                dd.plot()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

class Parser():
    def __init__(self):
        self.batch_size = 16
        self.test_batch_size = 1
        self.epochs = 100
        self.lr = 0.001
        self.gamma = 0.7
        self.log_interval = 1
        self.save_model = False
        

def main():
    # Training settings
    args = Parser()
    
    if True:
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    train_kwargs = {'batch_size': args.batch_size}
    test_kwargs = {'batch_size': args.test_batch_size}
    if device == torch.device("cuda"):
        cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
        train_kwargs.update(cuda_kwargs)
        test_kwargs.update(cuda_kwargs)

    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    dataset1 = datasets.MNIST('example_data', train=True, download=True,
                       transform=transform)
    dataset2 = datasets.MNIST('example_data', train=False,
                       transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
    test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

    model = DecentNet().to(device)
    optimizer = optim.Adadelta(model.parameters(), lr=args.lr)

    scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        scheduler.step()
        
        model.update()
        
        
        
        


    if args.save_model:
        torch.save(model.state_dict(), "mnist_cnn.ckpt")



# main()

In [ ]:
a = np.empty((1,81))

In [ ]:
a = np.full(81, np.nan)
geometry_array = a.reshape((9,9))

In [ ]:
geometry_array

In [ ]:
for i in model.parameters():
    print(i.shape)

In [ ]:
labels = torch.tensor([1, 0, 5, 2])
labels = labels.unsqueeze(0)

target = torch.zeros(labels.size(0), 10).scatter_(1, labels, 1.)
print(target)
print(labels)

In [ ]:
16*16

In [ ]:
#model.conv2.importance

In [ ]:
#model.conv2.importance.shape

In [ ]:
print(list(model.conv2.weight.shape))

# Lightning version

In [19]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
import os

class DecentLightning(pl.LightningModule):
    def __init__(self, model_kwargs):
        super().__init__()
        self.model = DecentNet().to("cuda")
        self.criterion = model_kwargs["criterion"]
        self.optimizer = model_kwargs["optimizer"]
        self.lr = model_kwargs["lr"]
        self.cc_weight = model_kwargs["cc_weight"]

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # 1       
        loss = self._loss_n_metrics(batch, mode="train")
        print("training_step")
        #loss = torch.tensor(1)
        return loss
    
    
    def validation_step(self):
        # 2
        print("validation_step")
        #self._loss_n_metrics(batch, mode="val")
    
    def on_validation_epoch_end(self):
        print("on_validation_epoch_end")
        # 3
        pass
    
    def on_train_epoch_end(self):
        print("on_validation_epoch_end")
        # 4
        pass

    def configure_optimizers(self):
        print("configure_optimizers")
        
        if self.optimizer == "adamw":
            optimizer = optim.AdamW(self.parameters(), lr=self.lr)
            lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150], gamma=0.1)
            return [optimizer], [lr_scheduler]
        else:
            return optim.SGD(self.parameters(), lr=self.lr)
        
    def get_cc(self):
        # compute connection cost
        # adapted from BIMT: https://github.com/KindXiaoming/BIMT/blob/main/mnist_3.5.ipynb
        # def get_cc(self, weight_factor=2.0, bias_penalize=True, no_penalize_last=False):
        # https://stackoverflow.com/questions/74086766/how-to-find-total-cost-of-each-path-in-graph-using-dictionary-in-python
        cc = 0
        """
        num_linear = len(self.linears)
        for i in range(num_linear):
            if i == num_linear - 1 and no_penalize_last:
                weight_factor = 0.
            biolinear = self.linears[i]
            dist = torch.sum(torch.abs(biolinear.out_coordinates.unsqueeze(dim=1) - biolinear.in_coordinates.unsqueeze(dim=0)),dim=2)
            cc += torch.mean(torch.abs(biolinear.linear.weight)*(weight_factor*dist+self.l0))
            if bias_penalize == True:
                cc += torch.mean(torch.abs(biolinear.linear.bias)*(self.l0))
        if self.token_embedding:
            cc += torch.mean(torch.abs(self.embedding)*(self.l0))
            #pass
        """
        return cc
    
    def _loss_n_metrics(self, batch, mode="train"):
        
        img, ground_truth = batch
        model_output = self(img.to("cuda")) # cause of the forward function
        
        ground_truth = ground_truth.unsqueeze(1)
        
        ground_truth_multi_hot = torch.zeros(ground_truth.size(0), 10).scatter_(1, ground_truth.to("cpu"), 1.)
        
        # this needs fixing
        # ground_truth_multi_hot = torch.zeros(ground_truth.size(0), 10).to("cuda").scatter_(torch.tensor(1).to("cuda"), ground_truth.to("cuda"), torch.tensor(1.).to("cuda")).to("cuda")
        
        loss = self.criterion(model_output, ground_truth_multi_hot.to("cuda"))
        cc = self.get_cc() * self.cc_weight
        # from BIMT
        # loss_train = loss_fn(mlp(x.to(device)), one_hots[label])
        # cc = mlp.get_cc(weight_factor=2.0, no_penalize_last=True)
        # total_loss = loss_train + lamb*cc
        
        
        acc = (model_output.argmax(dim=-1) == ground_truth).float().mean()

        self.log(f'{mode}_loss', loss, on_step=False, on_epoch=True)
        self.log(f'{mode}_cc', cc, on_step=False, on_epoch=True)
        self.log(f'{mode}_acc', acc, on_step=False, on_epoch=True)
        
        # loss + connection cost term
        return loss + cc



In [20]:
def dev_routine(**kwargs):
    
    train_kwargs = kwargs['train_kwargs']
    
    
    transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

    dataset1 = datasets.MNIST('example_data', train=True, download=True,
                       transform=transform)
    #dataset2 = datasets.MNIST('example_data', train=False,
    #                   transform=transform)
    train_dataloader = torch.utils.data.DataLoader(dataset1, batch_size=train_kwargs["batch_size"])
    #inference_loader = torch.utils.data.DataLoader(dataset2, **train_kwargs)
    
    print(train_kwargs)
    trainer = pl.Trainer(default_root_dir=os.path.join(train_kwargs["ckpt_path"], "example_results"),
                         accelerator="gpu" if str(train_kwargs["device"]).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=train_kwargs["epochs"],
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc"),
                                    LearningRateMonitor("epoch")])
    
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(train_kwargs["ckpt_path"], "DecentNet.ckpt")
    if os.path.isfile(pretrained_filename):
        print(f"Found pretrained model at {pretrained_filename}, loading...")
        model = DecentLightning.load_from_checkpoint(pretrained_filename) # Automatically loads the model with the saved hyperparameters
    else:
        pl.seed_everything(19) # To be reproducable
        
        # Initialize the LightningModule and LightningDataModule
        model = DecentLightning(kwargs['model_kwargs'])

        # Train the model using a Trainer
        trainer.fit(model, train_dataloader)
        
        model = DecentLightning.load_from_checkpoint(trainer.checkpoint_callback.best_model_path) # Load best checkpoint after training

    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    test_result = trainer.test(model, test_loader, verbose=False)
    result = {"test": test_result[0]["test_acc"], "val": val_result[0]["test_acc"]}

    return model, result
    

In [21]:
model, results = dev_routine(model_kwargs={
                                'embed_dim': 256,
                                'hidden_dim': 512,
                                'num_heads': 8,
                                'num_layers': 6,
                                'patch_size': 4,
                                'num_channels': 3,
                                'num_patches': 64,
                                'num_classes': 10,
                                'criterion': torch.nn.BCELoss(),
                                'optimizer': "adamw",
                                'lr': 0.001,
                                'cc_weight': 0.2,
                            },
                            train_kwargs={
                                'epochs': 100,
                                'batch_size': 16,
                                # 'test_batch_size': 1,
                                'ckpt_path': "",
                                'device': "cuda"
                            }
                            )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 19


{'epochs': 100, 'batch_size': 16, 'ckpt_path': '', 'device': 'cuda'}

[[nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan  6.  2. nan nan nan nan]
 [ 1. nan  7. nan nan nan nan nan  3.]
 [nan nan nan nan nan nan nan nan  0.]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan  4.]
 [nan nan nan nan nan nan nan  5. nan]]


[[ 5. nan nan nan  8. nan 12. nan nan]
 [nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan 10. nan nan nan]
 [nan  1. 11.  2. nan nan  6. nan nan]
 [nan  3. nan nan nan nan nan nan nan]
 [13. nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan  9. 15.]
 [nan nan nan nan nan nan nan nan nan]
 [ 7. nan nan nan  0. 14. nan nan  4.]]


[[nan nan nan nan nan 12. nan nan nan]
 [nan nan nan nan nan  1.  8.  2. nan]
 [nan nan  0. nan 20. 16. nan nan nan]
 [nan nan nan nan  5. nan nan nan nan]
 [nan nan  3. 11. nan 13. 14. 21. nan]
 [nan nan 15. nan nan nan  

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params
----------------------------------------
0 | model     | DecentNet | 7.0 K 
1 | criterion | BCELoss   | 0     
----------------------------------------
7.0 K     Trainable params
0         Non-trainable params
7.0 K     Total params
0.028     Total estimated model params size (MB)


configure_optimizers


Training: 0it [00:00, ?it/s]

training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
traini

C:\Users\Prinzessin\anaconda3\envs\feta\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:359: UserWarning: `ModelCheckpoint(monitor='val_acc')` could not find the monitored key in the returned metrics: ['train_loss', 'train_acc', 'epoch', 'step']. HINT: Did you call `log('val_acc', value)` in the `LightningModule`?
  warning_cache.warn(m)


training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
training_step
traini

C:\Users\Prinzessin\anaconda3\envs\feta\lib\site-packages\pytorch_lightning\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


PermissionError: [Errno 13] Permission denied: 'C:/Users/Prinzessin/projects/decentnet/datasceyence/examples'

# conv filter test

In [519]:
import torch
import torch.nn.functional as F

# this is one filter

w_groups = 1 # groups of the channels
w_channels = 1024 # input channels
w_filters = 10

assert w_filters % w_groups == 0
assert w_channels % w_groups == 0

# batch size x channels (= w_groups*w_channels) x width x height
inputs = torch.autograd.Variable(torch.randn(27,w_groups*w_channels,100,100))

# w_filters x w_channels x kernel x kernel
weights = torch.autograd.Variable(torch.randn(w_filters,w_channels,3,3))

# batch size x w_filters x width x height
out = F.conv2d(inputs, weights, padding=1, groups=w_groups)

import random



print(inputs.shape, "- batch x groups*channels x width x height")
print(weights.shape, "- filters x channels x kernel x kernel")
print(out.shape, "- batch x filters x width x height")
print()

print("*"*50)

# batch size x channels (= w_groups*w_channels) x width x height
inputs = torch.autograd.Variable(torch.randn(27,w_groups*w_channels,100,100))

output_list = []

# for each filter, we need different true false vales for our channels
active = list(np.random.choice([True, False], size=w_channels, replace=True, p=None))

# w_filters x w_channels x kernel x kernel
weights = torch.autograd.Variable(torch.randn(1,w_channels,3,3))

import time



output_list = []
start = time.time()
for i in range (10):
    for _ in range(w_filters):
        
        pass

        #print()
        #print(active)
        #print()
        #print(inputs.shape, "- batch x groups*channels x width x height")
        #print(weights.shape, "- filters x channels x kernel x kernel")


        # need to remove weight and input channels according to active list for each filter
        #print(inputs.shape)
        #print(inputs[:,active,:,:].shape)

        #print(weights.shape)
        #print(weights[:,active,:,:].shape)




        # batch size x w_filters x width x height
        

        
            



            #output_list.append(this_output)
        #print(this_output.shape, "- batch x 1 filter x width x height")

    #out = torch.cat(output_list, dim=1)

end = time.time()
print(end - start)



weights = torch.autograd.Variable(torch.randn(w_filters,w_channels,3,3))


start = time.time()
for i in range (10):
    i_tmp = inputs[:,active,:,:]
    w_tmp = weights[:,active,:,:]
    this_output = F.conv2d(i_tmp, w_tmp, padding=1, groups=w_groups)
end = time.time()
print(end - start)




weights = torch.autograd.Variable(torch.randn(w_filters,w_channels,3,3))

start = time.time()
for i in range (10):
    this_output = F.conv2d(inputs, weights, padding=1, groups=w_groups)
end = time.time()
print(end - start)



print()
print(out.shape, "- batch x filters x width x height")
print()
print()






# take the mean of all - we can remove all sorts of information from the out tensor
mean = torch.mean(out, 1, keepdim=True)
print(mean.shape, "- mean accross the filters (no sense here ...)")

torch.Size([27, 1024, 100, 100]) - batch x groups*channels x width x height
torch.Size([10, 1024, 3, 3]) - filters x channels x kernel x kernel
torch.Size([27, 10, 100, 100]) - batch x filters x width x height

**************************************************
0.0


C:\Users\Christina\AppData\Local\Temp\ipykernel_12532\2723409993.py:95: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  i_tmp = inputs[:,active,:,:]
C:\Users\Christina\AppData\Local\Temp\ipykernel_12532\2723409993.py:96: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  w_tmp = weights[:,active,:,:]


3.23818039894104
3.1647353172302246

torch.Size([27, 10, 100, 100]) - batch x filters x width x height


torch.Size([27, 1, 100, 100]) - mean accross the filters (no sense here ...)


In [ ]:
import sys
import torch

#b = torch.empty(w_filters, w_channels, dtype=torch.bool)
b = torch.ByteTensor(500, w_channels)
print(sys.getsizeof(b.storage())) # 1310776 (bytes)

#a = torch.empty(w_filters, w_channels, dtype=torch.uint8)
a = torch.ByteTensor(500, w_channels)
print(sys.getsizeof(a.storage())) # 1310776 (bytes)

active = list(np.random.choice([True, False], size=w_channels, replace=True, p=None))
print(sys.getsizeof(active)*500) # 1310776 (bytes)

weights = torch.FloatTensor(torch.randn(500,w_channels,3,3))
print(sys.getsizeof(weights.storage())) # 1310776 (bytes) 36912
print(36912*500)

In [ ]:
weights

In [ ]:
np.random.randint(2, size=10)

list(np.random.choice([True, False], size=10, replace=True, p=None))

In [ ]:
np.random.randint(1, 82, size=200)

In [ ]:
9*9

In [ ]:
np.random.choice(a=4, size=2, replace=False)

In [ ]:
import torch
import torch.nn.functional as F

# this is one filter

w_groups = 27 # groups of the channels
w_channels = 7 # input channels
w_filters = 200
batch_size = 27

# batch size x channels (= w_groups*w_channels) x width x height
inputs = torch.autograd.Variable(torch.randn(batch_size, w_channels*w_groups, 100,100))
#inputs = torch.autograd.Variable(torch.randn(w_channels*w_groups, 100,100))

# w_groups x w_channels x kernel x kernel
weights = torch.autograd.Variable(torch.randn(1, w_channels,3,3))
#weights = torch.autograd.Variable(torch.randn(w_groups*w_channels,3,3))

print(inputs.shape, "- batch x groups*channels x width x height")
print(weights.shape, "- filter 1 x channels x kernel x kernel")

try:
    o_list = []
    for _ in range(w_filters):
        # batch size x groups x width x height
        out = F.conv2d(inputs, weights, groups=w_groups)
        o_list.append(out)
        # take the mean of all - we can remove all sorts of information from the out tensor
        #mean = torch.mean(out, 1, keepdim=True)
    
    print(torch.cat(o_list, dim=1).shape, "- batch x filters x width x height")
    #print(mean.shape, "- mean accross the groups")
except Exception as e:
    print(e)



In [ ]:
filters = torch.randn(8, 4, 3, 3)
inputs = torch.randn(1, 4, 5, 5)
F.conv2d(inputs, filters, padding=1)

# Visualise filters and channels

In [ ]:
model = Net() # .to("cuda")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import utils

def visChannels(tensor, ch=0, allkernels=False, nrow=8, padding=1): 
    n,c,w,h = tensor.shape

    if allkernels: tensor = tensor.view(n*c, -1, w, h)
    elif c != 3: tensor = tensor[:,ch,:,:].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    
    plt.figure(figsize=(nrow,rows) )
    plt.title(f"Channels with index {ch}")
    plt.imshow(grid.detach().cpu().numpy().transpose((1, 2, 0)))


def visFilters(tensor, filt=0, allkernels=False, nrow=8, padding=1): 
    f,c,w,h = tensor.shape

    if allkernels: tensor = tensor.view(f*c, -1, w, h)
    elif f != 3: tensor = tensor[filt,:,:,:].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    plt.figure( figsize=(nrow,rows) )
    plt.title(f"Filter {filt}")
    plt.imshow(grid.detach().cpu().numpy().transpose((1, 2, 0)))

def visFilters_subplot(subplot, tensor, filt=0, allkernels=False, nrow=8, padding=1): 
    f,c,w,h = tensor.shape

    if allkernels: tensor = tensor.view(f*c, -1, w, h)
    elif f != 3: tensor = tensor[filt,:,:,:].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    # plt.figure( figsize=(nrow,rows) )
    subplot.set_title(f"Filter {filt+1} with {c} channels")
    subplot.imshow(grid.detach().cpu().numpy().transpose((1, 2, 0)))
    subplot.axis('off')
    
layer = 1
filter = model.conv2.weight.data.clone()

print(model.conv2.weight.shape)

# need to match the network parameters!!!!
in_channels = 5
out_filters = 3 # 64


fig, subplot = plt.subplots(out_filters, figsize=(10, 10))
fig.suptitle(f'Layer with shape {list(model.conv2.weight.shape)} [out, in, kernel, kernel]')

for filt in range(0, out_filters):
    
    visFilters_subplot(subplot[filt], filter, filt=filt, allkernels=False)

    #plt.axis('off')
    #plt.ioff()
    
plt.tight_layout()
plt.savefig(f"example_results/filter_with_weights.png")
plt.show()
    
if False:    
    for filt in range(0, out_filters):

        visFilters(filter, filt=filt, allkernels=False)

        plt.axis('off')
        plt.ioff()
        plt.tight_layout()
        #plt.savefig(f"examples/example_results/filter_with_weights.png")
        plt.show()

    for ch in range(0, in_channels):

        visChannels(filter, ch=ch, allkernels=False)

        plt.axis('off')
        plt.ioff()
        plt.show()

In [ ]:
import torchvision
res = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)

In [ ]:
res.layer1[0].conv1.bias == None

In [ ]:
res.layer1[0].conv1.weight.shape

In [ ]:
res.extra_repr